<a href="https://colab.research.google.com/github/AkhilRam7/Projects/blob/main/Gesture_Recognition_Conv3d_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
!gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
!unzip Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [ ]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os

from imageio import imread
from skimage.transform import resize
from skimage.util import crop

import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())


train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())

batch_size = 30
frames = 16
image_height = 160
image_width = 160
crop_h = 0
crop_w = 0


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    print('crop=',(crop_w,crop_h),'; frames=',frames,'; image:',(image_width, image_height))
    img_idx = np.round(np.linspace(0,29, frames)).astype(int)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. 
                    croped_image = image[crop_h:image.shape[0]-crop_h,crop_w:image.shape[1]-crop_w]

                    #Resize
                    image_resized = resize(croped_image, (image_height, image_width))
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        if len(t) != num_batches*batch_size:
            num_batches = len(t)//batch_size # calculate the number of batches
            rem = len(t) - num_batches*batch_size
            for batch in range(1): # we iterate over the number of batches
                batch_data = np.zeros((rem,len(img_idx),image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((rem,5)) # batch_labels is the one hot representation of the output
              
                for folder in range(rem): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        #crop the images and resize them. 
                        croped_image = image[crop_h:image.shape[0]-crop_h,crop_w:image.shape[0]-crop_w]

                        #Resize
                        image_resized = resize(croped_image, (image_height, image_width))

                        batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = (image_resized[:,:,0])/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = (image_resized[:,:,0])/255 #normalise and feed in the image

                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

#write your model here

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 16, 160, 160, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 16, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 80, 80, 16)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 8, 80, 80, 32)     4128      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 80, 80, 32)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 80, 80, 32)     1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  datasets/Project_data/train ; batch size = 30
crop= (0, 0) ; frames= 16 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.5812 - categorical_accuracy: 0.4178Source path =  datasets/Project_data/val ; batch size = 30
crop= (0, 0) ; frames= 16 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0404_58_19.362885/model-00001-1.58122-0.41780-3.73319-0.16000.h5
23/23 [==============================] - 86s 4s/step - loss: 1.5812 - categorical_accuracy: 0.4178 - val_loss: 3.7332 - val_categorical_accuracy: 0.1600
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 0.9829 - categorical_accuracy: 0.6440
Epoch 00002: saving model to model_init_2021-07-0404_58_19.362885/model-00002-0.98290-0.64404-5.71050-0.21000.h5
23/23 [==============================] - 88s 4s/step - loss: 0.9829 - categorical_accuracy: 0.6440 - val_loss: 5.7105 - v

# Experiment Models

## Model 1

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = No Crop

**Frames** = 30


In [ ]:
batch_size = 30
frames = 30
image_height = 160
image_width = 160
epoch = 3
crop_h = 0
crop_w = 0

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_4 (Activation)    (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_5 (Activation)    (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (0, 0) ; frames= 30 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.3925 - categorical_accuracy: 0.4947Source path =  datasets/Project_data/val ; batch size = 30
crop= (0, 0) ; frames= 30 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0404_58_19.362885/model-00001-1.39254-0.49472-2.68806-0.21000.h5
23/23 [==============================] - 164s 7s/step - loss: 1.3925 - categorical_accuracy: 0.4947 - val_loss: 2.6881 - val_categorical_accuracy: 0.2100
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 0.8719 - categorical_accuracy: 0.6802
Epoch 00002: saving model to model_init_2021-07-0404_58_19.362885/model-00002-0.87192-0.68024-4.67934-0.22000.h5
23/23 [==============================] - 162s 7s/step - loss: 0.8719 - categorical_accuracy: 0.6802 - val_loss: 4.6793 - val_categorical_accuracy: 0.2200
Epoch 3/15
23/23 [========================

## Model 2

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = No Crop

**Frames** = 25


In [ ]:
batch_size = 30
frames = 25
image_height = 160
image_width = 160
epoch = 15
crop_h = 0
crop_w = 0

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 25, 160, 160, 16)  1312      
_________________________________________________________________
activation_8 (Activation)    (None, 25, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 25, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 12, 80, 80, 16)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 12, 80, 80, 32)    4128      
_________________________________________________________________
activation_9 (Activation)    (None, 12, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 12, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (0, 0) ; frames= 25 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.4314 - categorical_accuracy: 0.4419Source path =  datasets/Project_data/val ; batch size = 30
crop= (0, 0) ; frames= 25 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0414_02_12.767239/model-00001-1.43139-0.44193-2.68603-0.18000.h5
23/23 [==============================] - 135s 6s/step - loss: 1.4314 - categorical_accuracy: 0.4419 - val_loss: 2.6860 - val_categorical_accuracy: 0.1800
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 0.9712 - categorical_accuracy: 0.6244
Epoch 00002: saving model to model_init_2021-07-0414_02_12.767239/model-00002-0.97116-0.62443-4.25928-0.18000.h5
23/23 [==============================] - 136s 6s/step - loss: 0.9712 - categorical_accuracy: 0.6244 - val_loss: 4.2593 - val_categorical_accuracy: 0.1800
Epoch 3/15
23/23 [========================

## Model 3

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = No Crop

**Frames** = 20


In [ ]:
batch_size = 30
frames = 20
image_height = 160
image_width = 160
epoch = 15
crop_h = 0
crop_w = 0

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 20, 160, 160, 16)  1312      
_________________________________________________________________
activation_12 (Activation)   (None, 20, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 20, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 10, 80, 80, 16)    0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 10, 80, 80, 32)    4128      
_________________________________________________________________
activation_13 (Activation)   (None, 10, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 10, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (0, 0) ; frames= 20 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.5376 - categorical_accuracy: 0.4329Source path =  datasets/Project_data/val ; batch size = 30
crop= (0, 0) ; frames= 20 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0414_02_12.767239/model-00001-1.53759-0.43288-3.13030-0.16000.h5
23/23 [==============================] - 108s 5s/step - loss: 1.5376 - categorical_accuracy: 0.4329 - val_loss: 3.1303 - val_categorical_accuracy: 0.1600
Epoch 2/15
22/23 [===========================>..] - ETA: 3s - loss: 1.0260 - categorical_accuracy: 0.6091
Epoch 00002: saving model to model_init_2021-07-0414_02_12.767239/model-00002-1.03272-0.60633-4.74658-0.16000.h5
23/23 [==============================] - 109s 5s/step - loss: 1.0327 - categorical_accuracy: 0.6063 - val_loss: 4.7466 - val_categorical_accuracy: 0.1600
Epoch 3/15
23/23 [========================

## Insights

Model | Batch Size | Frames | Accuracy | Parameters
--- | --- | -- | -- | -- 
Model 1 | 30 | 30 | 0.61 | 1,736,389
Model 2 | 30 | 25 | 0.63 | 1,736,389
Model 3 | 30 | 20 | 0.65 | 1,736,389



We can see that model with less frames tends to reach accuracy sooner compared to other models.

## Model 4

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = (5,5)

**Frames** = 20


In [ ]:
batch_size = 30
frames = 20
image_height = 160
image_width = 160
epoch = 15
crop_h = 5
crop_w = 5

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 20, 160, 160, 16)  1312      
_________________________________________________________________
activation_16 (Activation)   (None, 20, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 20, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 10, 80, 80, 16)    0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 10, 80, 80, 32)    4128      
_________________________________________________________________
activation_17 (Activation)   (None, 10, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 10, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (5, 5) ; frames= 20 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.5690 - categorical_accuracy: 0.4223Source path =  datasets/Project_data/val ; batch size = 30
crop= (5, 5) ; frames= 20 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0414_02_12.767239/model-00001-1.56901-0.42232-1.98571-0.16000.h5
23/23 [==============================] - 106s 5s/step - loss: 1.5690 - categorical_accuracy: 0.4223 - val_loss: 1.9857 - val_categorical_accuracy: 0.1600
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 0.9689 - categorical_accuracy: 0.6305
Epoch 00002: saving model to model_init_2021-07-0414_02_12.767239/model-00002-0.96895-0.63047-4.08939-0.10000.h5
23/23 [==============================] - 104s 5s/step - loss: 0.9689 - categorical_accuracy: 0.6305 - val_loss: 4.0894 - val_categorical_accuracy: 0.1000
Epoch 3/15
23/23 [========================

## Model 5

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = (10,10)

**Frames** = 20


In [ ]:
batch_size = 30
frames = 20
image_height = 160
image_width = 160
epoch = 15
crop_h = 10
crop_w = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_20 (Conv3D)           (None, 20, 160, 160, 16)  1312      
_________________________________________________________________
activation_20 (Activation)   (None, 20, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 20, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 10, 80, 80, 16)    0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 10, 80, 80, 32)    4128      
_________________________________________________________________
activation_21 (Activation)   (None, 10, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 10, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (10, 10) ; frames= 20 ; image: (160, 160)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.5534 - categorical_accuracy: 0.4253Source path =  datasets/Project_data/val ; batch size = 30
crop= (10, 10) ; frames= 20 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0414_02_12.767239/model-00001-1.55343-0.42534-5.00134-0.16000.h5
23/23 [==============================] - 104s 5s/step - loss: 1.5534 - categorical_accuracy: 0.4253 - val_loss: 5.0013 - val_categorical_accuracy: 0.1600
Epoch 2/15
23/23 [==============================] - ETA: 0s - loss: 0.9223 - categorical_accuracy: 0.6425
Epoch 00002: saving model to model_init_2021-07-0414_02_12.767239/model-00002-0.92232-0.64253-7.55406-0.21000.h5
23/23 [==============================] - 105s 5s/step - loss: 0.9223 - categorical_accuracy: 0.6425 - val_loss: 7.5541 - val_categorical_accuracy: 0.2100
Epoch 3/15
23/23 [====================

## Model 6

**Batchsize** = 30 

**Image Size** = (160,160,3)

**Crop** = (10,10)

**Frames** = 20


In [ ]:
batch_size = 30
frames = 20
image_height = 160
image_width = 160
epoch = 3
crop_h = 20
crop_w = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 20, 160, 160, 16)  1312      
_________________________________________________________________
activation_24 (Activation)   (None, 20, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 20, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 10, 80, 80, 16)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 80, 80, 32)    4128      
_________________________________________________________________
activation_25 (Activation)   (None, 10, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_37 (Batc (None, 10, 80, 80, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
crop= (20, 20) ; frames= 20 ; image: (160, 160)
Epoch 1/5
23/23 [==============================] - ETA: 0s - loss: 1.5096 - categorical_accuracy: 0.3967Source path =  datasets/Project_data/val ; batch size = 30
crop= (20, 20) ; frames= 20 ; image: (160, 160)

Epoch 00001: saving model to model_init_2021-07-0404_58_19.362885/model-00001-1.50964-0.39668-3.44541-0.16000.h5
23/23 [==============================] - 105s 5s/step - loss: 1.5096 - categorical_accuracy: 0.3967 - val_loss: 3.4454 - val_categorical_accuracy: 0.1600
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 1.0242 - categorical_accuracy: 0.6154
Epoch 00002: saving model to model_init_2021-07-0404_58_19.362885/model-00002-1.02420-0.61538-5.54630-0.11000.h5
23/23 [==============================] - 104s 5s/step - loss: 1.0242 - categorical_accuracy: 0.6154 - val_loss: 5.5463 - val_categorical_accuracy: 0.1100
Epoch 3/5
23/23 [=======================

## Insights

Model | Batch Size | Frames | Crop | Accuracy | Parameters
--- | --- | -- | -- | -- | --
Model 1 | 30 | 30 | (0,0)|0.61 | 1,736,389
Model 2 | 30 | 25 | (0,0)|0.63 | 1,736,389
Model 3 | 30 | 20 | (0,0)|0.65 | 1,736,389
Model 4 | 30 | 20 | (5,5)|0.60 | 1,736,389
Model 5 | 30 | 20 | (10,10)|0.60 | 1,736,389
Model 6 | 30 | 20 | (15,15)|0.59 | 1,736,389

Cropping a picture does affect the accuracy and its looking idle to crop as less as possible


## Model 7

**Batchsize** = 30 

**Image Size** = (200,200,3)

**Crop** = (10,10)

**Frames** = 20


In [ ]:
batch_size = 30
frames = 20
image_height = 200
image_width = 200
epoch = 3
crop_h = 10
crop_w = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_28 (Conv3D)           (None, 20, 200, 200, 16)  1312      
_________________________________________________________________
activation_28 (Activation)   (None, 20, 200, 200, 16)  0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 20, 200, 200, 16)  64        
_________________________________________________________________
max_pooling3d_28 (MaxPooling (None, 10, 100, 100, 16)  0         
_________________________________________________________________
conv3d_29 (Conv3D)           (None, 10, 100, 100, 32)  4128      
_________________________________________________________________
activation_29 (Activation)   (None, 10, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_43 (Batc (None, 10, 100, 100, 32) 

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
crop= (10, 10) ; frames= 20 ; image: (200, 200)
Epoch 1/15
23/23 [==============================] - ETA: 0s - loss: 1.7058 - categorical_accuracy: 0.3786Source path =  Project_data/val ; batch size = 30
crop= (10, 10) ; frames= 20 ; image: (200, 200)
23/23 [==============================] - 225s 8s/step - loss: 1.6976 - categorical_accuracy: 0.3809 - val_loss: 2.8649 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2021-07-0415_00_14.300540/model-00001-1.50869-0.43439-2.86485-0.16000.h5
Epoch 2/15
23/23 [==============================] - 181s 8s/step - loss: 1.1559 - categorical_accuracy: 0.5175 - val_loss: 5.1487 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2021-07-0415_00_14.300540/model-00002-1.11692-0.55354-5.14867-0.21000.h5
Epoch 3/15
23/23 [==============================] - 178s 8s/step - loss: 1.0538 - categorical_accuracy: 0.5929 - val_loss: 6.2643 - val_categorical_ac

## Model 8

**Batchsize** = 40 

**Image Size** = (140,140,3)

**Crop** = (10,10)

**Frames** = 20


In [ ]:
batch_size = 40
frames = 20
image_height = 140
image_width = 140
epoch = 3
crop_h = 10
crop_w = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_32 (Conv3D)           (None, 20, 140, 140, 16)  1312      
_________________________________________________________________
activation_32 (Activation)   (None, 20, 140, 140, 16)  0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 20, 140, 140, 16)  64        
_________________________________________________________________
max_pooling3d_32 (MaxPooling (None, 10, 70, 70, 16)    0         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 10, 70, 70, 32)    4128      
_________________________________________________________________
activation_33 (Activation)   (None, 10, 70, 70, 32)    0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 10, 70, 70, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
crop= (10, 10) ; frames= 20 ; image: (140, 140)
Epoch 1/15
17/17 [==============================] - ETA: 0s - loss: 1.7941 - categorical_accuracy: 0.3376Source path =  Project_data/val ; batch size = 40
crop= (10, 10) ; frames= 20 ; image: (140, 140)
17/17 [==============================] - 147s 9s/step - loss: 1.7850 - categorical_accuracy: 0.3396 - val_loss: 1.8895 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2021-07-0415_00_14.300540/model-00001-1.63011-0.37255-1.88948-0.29000.h5
Epoch 2/15
17/17 [==============================] - 139s 9s/step - loss: 1.1167 - categorical_accuracy: 0.5546 - val_loss: 2.6611 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2021-07-0415_00_14.300540/model-00002-1.09443-0.56712-2.66111-0.23000.h5
Epoch 3/15
17/17 [==============================] - 137s 9s/step - loss: 0.9353 - categorical_accuracy: 0.6547 - val_loss: 3.9608 - val_categorical_ac

## Model 9

**Batchsize** = 60 

**Image Size** = (100,100,3)

**Crop** = (10,10)

**Frames** = 20


In [ ]:
batch_size = 60
frames = 20
image_height = 100
image_width = 100
epoch = 3
crop_h = 10
crop_w = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_36 (Conv3D)           (None, 20, 100, 100, 16)  1312      
_________________________________________________________________
activation_36 (Activation)   (None, 20, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 20, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_36 (MaxPooling (None, 10, 50, 50, 16)    0         
_________________________________________________________________
conv3d_37 (Conv3D)           (None, 10, 50, 50, 32)    4128      
_________________________________________________________________
activation_37 (Activation)   (None, 10, 50, 50, 32)    0         
_________________________________________________________________
batch_normalization_55 (Batc (None, 10, 50, 50, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 60
crop= (10, 10) ; frames= 20 ; image: (100, 100)
Epoch 1/15
12/12 [==============================] - ETA: 0s - loss: 2.0815 - categorical_accuracy: 0.2911Source path =  Project_data/val ; batch size = 60
crop= (10, 10) ; frames= 20 ; image: (100, 100)
12/12 [==============================] - 138s 12s/step - loss: 2.0663 - categorical_accuracy: 0.2945 - val_loss: 1.7033 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2021-07-0415_00_14.300540/model-00001-1.88431-0.33484-1.70334-0.21000.h5
Epoch 2/15
12/12 [==============================] - 129s 12s/step - loss: 1.3178 - categorical_accuracy: 0.5192 - val_loss: 2.4379 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2021-07-0415_00_14.300540/model-00002-1.25966-0.51131-2.43789-0.21000.h5
Epoch 3/15
12/12 [==============================] - 136s 12s/step - loss: 1.1205 - categorical_accuracy: 0.5723 - val_loss: 3.6002 - val_categorical

## Insights

Model | Batch Size | Frames | Crop | Image | Accuracy | Parameters
--- | --- | -- | -- | -- | -- | --
Model 1 | 30 | 30 | (0,0)| (160,160)|0.61 | 1,736,389
Model 2 | 30 | 25 | (0,0)| (160,160)|0.63 | 1,736,389
Model 3 | 30 | 20 | (0,0)| (160,160)|0.65 | 1,736,389
Model 4 | 30 | 20 | (5,5)| (160,160)|0.60 | 1,736,389
Model 5 | 30 | 20 | (10,10)| (160,160)|0.60 | 1,736,389
Model 6 | 30 | 20 | (15,15)| (160,160)|0.59 | 1,736,389
Model 7 | 30 | 20 | (10,10)| (200,200)|0.66 | 2,457,285
Model 8 | 40 | 20 | (10,10)| (140,140)|0.61 | 1,146,565
Model 9 | 60 | 20 | (10,10)| (100,100)|0.57 | 687,813

When the image is reduced the accuaracy is coming down but still we have less parameters which is an advantage


## Model 10

**Batchsize** = 40 

**Image Size** = (120,120,3)

**Crop** = (10,10)

**Frames** = 20

In [ ]:
batch_size = 40
frames = 20
image_height = 120
image_width = 120
epoch = 5
crop_h = 10
crop_w = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(8, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(16, (3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))


optimiser = optimizers.Adam(lr=0.0002) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 20, 120, 120, 8)   656       
_________________________________________________________________
activation_4 (Activation)    (None, 20, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 20, 120, 120, 8)   32        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 60, 60, 8)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 10, 60, 60, 16)    3472      
_________________________________________________________________
activation_5 (Activation)    (None, 10, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 10, 60, 60, 16)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epoch, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
crop= (10, 10) ; frames= 20 ; image: (120, 120)
Epoch 1/5
17/17 [==============================] - ETA: 0s - loss: 2.3750 - categorical_accuracy: 0.2088Source path =  Project_data/val ; batch size = 40
crop= (10, 10) ; frames= 20 ; image: (120, 120)
17/17 [==============================] - 178s 8s/step - loss: 2.3656 - categorical_accuracy: 0.2117 - val_loss: 1.6106 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2021-07-0412_32_42.877101/model-00001-2.20478-0.26094-1.61057-0.24000.h5
Epoch 2/5
17/17 [==============================] - 135s 8s/step - loss: 1.6508 - categorical_accuracy: 0.3649 - val_loss: 1.6465 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2021-07-0412_32_42.877101/model-00002-1.65224-0.37255-1.64649-0.22000.h5
Epoch 3/5
17/17 [==============================] - 146s 9s/step - loss: 1.3901 - categorical_accuracy: 0.4608 - val_loss: 1.7822 - val_categorical_accur

## Model 11

**Batchsize** = 20 

**Image Size** = (100,100,3)

**Crop** = (5,5)

**Frames** = 20

In [ ]:
batch_size = 20
frames = 20
image_height = 100
image_width = 100
epoch = 30
crop_h = 5
crop_w = 5

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:

model = Sequential()
model.add(Conv3D(16, (3, 3, 3), padding='same', 
                 input_shape=(frames, image_height, image_width,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Conv3D(32, (2,2,2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, (2, 2, 2), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(5,activation='softmax'))


optimiser = optimizers.Adam(lr=0.0002) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 20, 100, 100, 16)  1312      
_________________________________________________________________
activation_4 (Activation)    (None, 20, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 20, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 50, 50, 16)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 10, 50, 50, 32)    4128      
_________________________________________________________________
activation_5 (Activation)    (None, 10, 50, 50, 32)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 10, 50, 50, 32)   

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epoch, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
34/34 [==============================] - ETA: 0s - loss: 1.5818 - categorical_accuracy: 0.4118Source path =  datasets/Project_data/val ; batch size = 20
crop= (5, 5) ; frames= 20 ; image: (100, 100)

Epoch 00001: saving model to model_init_2021-07-0414_02_12.767239/model-00001-1.58180-0.41176-1.94502-0.16000.h5
34/34 [==============================] - 98s 3s/step - loss: 1.5818 - categorical_accuracy: 0.4118 - val_loss: 1.9450 - val_categorical_accuracy: 0.1600
Epoch 2/30
34/34 [==============================] - ETA: 0s - loss: 1.0402 - categorical_accuracy: 0.6018
Epoch 00002: saving model to model_init_2021-07-0414_02_12.767239/model-00002-1.04020-0.60181-3.46364-0.13000.h5
34/34 [==============================] - 95s 3s/step - loss: 1.0402 - categorical_accuracy: 0.6018 - val_loss: 3.4636 - val_categorical_accuracy: 0.1300
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 0.7782 - categorical_accuracy: 0.6968
Epoch 00003: saving model to model_init_2021-